# Actividad Guiada 1 de Algoritmos de Optimizacion

# Nombre: Sergio Moya Rodrigo

Algoritmos AG1 Sergio_Moya.ipynb



https://github.com/smoyarodrigo/03MAIR-Algoritmos-de-Optimizacion


# Algoritmo de Euclides para calcular el máximo común divisor (MCD)

In [1]:
def euclides(a, b):
    """
    Calcula el máximo común divisor (MCD) de dos números enteros a y b
    usando el Algoritmo de Euclides.
    """
    while b != 0:
        a, b = b, a % b
    return a

# Ejemplo de uso
num1 = int(input("Introduce el primer número: "))
num2 = int(input("Introduce el segundo número: "))

mcd = euclides(num1, num2)
print(f"El MCD de {num1} y {num2} es: {mcd}")



euclides(24, 12)

Introduce el primer número:  1025666
Introduce el segundo número:  12566


El MCD de 1025666 y 12566 es: 2


12

# Decorador python para medir el tiempo de ejecución

In [3]:
import time

# Decorador para medir el tiempo de ejecución
def medir_tiempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.perf_counter()
        resultado = func(*args, **kwargs)
        fin = time.perf_counter()
        print(f"Tiempo de ejecución de '{func.__name__}': {fin - inicio:.6f} segundos")
        return resultado
    return wrapper

# Método de Herón para aproximar la raiz cuadrada

In [5]:
@medir_tiempo
def raiz_cuadrada_heron(n, tolerancia=1e-10, max_iteraciones=1000):
    """
    Calcula la raíz cuadrada de un número n utilizando el Método de Herón.
    Parámetros:
    - n: Número del cual calcular la raíz cuadrada (debe ser >= 0).
    - tolerancia: Precisión deseada para la solución.
    - max_iteraciones: Número máximo de iteraciones permitidas.

    Retorna:
    - Una aproximación de la raíz cuadrada de n.
    """
    if n < 0:
        raise ValueError("No se puede calcular la raíz cuadrada de un número negativo.")

    # Inicialización de la estimación (puede ser n o un valor aproximado inicial)
    x = n if n != 0 else 0.0
    iteraciones = 0

    while iteraciones < max_iteraciones:
        # Nueva estimación según el método de Herón
        nuevo_x = 0.5 * (x + n / x)

        # Si la diferencia entre iteraciones es menor que la tolerancia, detenerse
        if abs(nuevo_x - x) < tolerancia:
            return nuevo_x

        x = nuevo_x
        iteraciones += 1

    # Si no se alcanzó la tolerancia en el número máximo de iteraciones
    raise RuntimeError(f"No se alcanzó la convergencia después de {max_iteraciones} iteraciones.")

# Ejemplo de uso
numero = float(input("Introduce el número para calcular su raíz cuadrada: "))
resultado = raiz_cuadrada_heron(numero)
print(f"La raíz cuadrada de {numero} es aproximadamente {resultado}")



Introduce el número para calcular su raíz cuadrada:  2


Tiempo de ejecución de 'raiz_cuadrada_heron': 0.000012 segundos
La raíz cuadrada de 2.0 es aproximadamente 1.414213562373095


# Algoritmos de Optimización. Componentes

<img src="Algoritmos de optimización.png">

# Variables decisoras

In [11]:
from itertools import product

# Paso 1: Inicializamos los datos
# Demanda mínima de autobuses por tramo
demanda = [4, 8, 10, 7, 12, 4]  # d[0], d[1], ..., d[5]
tramos = len(demanda)  # Número de tramos (6 en este caso)


# Restricciones

In [24]:
#Posible Solucion
x = [4,5,6,7,8,9]

for t in range(tramos):
    # Calculamos el número actual de autobuses que están cubriendo el tramo t
    cobertura_actual = x[t] + x[t - 1]  # Autobuses en t y t-1 (cíclico)

    # Si la cobertura actual es menor que la demanda, añadimos autobuses en t
    if cobertura_actual < demanda[t]:
        # Añadimos los autobuses necesarios en el tramo t
        x[t] += demanda[t] - cobertura_actual

# Función Objetivo

<img src="Captura de pantalla 2024-12-21 100340.png">

In [26]:
#Función objetivo
f_objetivo = sum(x)

# Determinar posibles soluciones y elegir la mejor así como determinar el orden de complejidad si aumenta la franja horaria

# A medida que se incrementan las franjas horarias, el tamaño del espacio de búsqueda y la complejidad computacional crecen de manera exponencial, ya que hay más combinaciones posibles. Esto sugiere que el problema es de tipo NP-difícil.
# Voy a implementar una solución utilizando programación lineal con la biblioteca pulp para encontrar la mejor solución, y calcularé el orden de complejidad teórico si las franjas horarias aumentan.

In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize

# Parámetros del problema
# Número de franjas horarias y demanda de autobuses por franja
franjas = [4, 8, 7, 8, 12, 12, 8, 4]
num_franjas = len(franjas)

# Crear el problema de minimización
problema = LpProblem("Minimizar_Autobuses", LpMinimize)

# Variables de decisión: x[i][j] indica si el autobús j opera en la franja i
num_autobuses = sum(franjas)  # Máximo posible en el peor caso
x = [[LpVariable(f"x_{i}_{j}", 0, 1, cat="Binary") for j in range(num_autobuses)] for i in range(num_franjas)]

# Función objetivo: minimizar el número total de autobuses
problema += lpSum(x[i][j] for i in range(num_franjas) for j in range(num_autobuses))

# Restricciones
for i in range(num_franjas):
    # Satisfacer la demanda de autobuses en cada franja
    problema += lpSum(x[i][j] for j in range(num_autobuses)) >= franjas[i]

for j in range(num_autobuses):
    # Un autobús no puede operar más de 8 horas consecutivas (2 franjas contiguas)
    for i in range(num_franjas - 2):
        problema += lpSum(x[k][j] for k in range(i, i + 3)) <= 2

# Resolver el problema
problema.solve()

# Resultado
print("Estado de la solución:", problema.status)
print("Número mínimo de autobuses requeridos:", sum(1 for j in range(num_autobuses) if any(x[i][j].varValue for i in range(num_franjas))))

# Visualizar la asignación
for j in range(num_autobuses):
    horarios = [i for i in range(num_franjas) if x[i][j].varValue == 1]
    if horarios:
        print(f"Autobús {j + 1}: opera en las franjas {horarios}")

# Evaluar la complejidad del problema
import math

# Complejidad teórica: O(2^(n*m)), donde n es el número de franjas y m es el número máximo de autobuses
n = num_franjas
m = num_autobuses
complejidad = math.pow(2, n * m)
print(f"Orden de complejidad (estimado): O(2^({n}*{m})) = O({complejidad:.2e})")
